In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from itertools import chain

import pandas as pd
import numpy as np
import time
import random


import re

In [2]:
# 这里的cookie换成你自己的cookie
#cookies = {
#    'JSESSIONID': 'D12AD85D6F706A7D3E03FC9C6C81035D',
    #'_pk_id.7.eda7': 'fbfbdec1aa1b4c42.163xxxxx.',
#}
#import requests

cookies = {
    'JSESSIONID': 'D12AD85D6F706A7D3E03FC9C6C81035D',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0',
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Origin': 'http://www.iachina.cn',
    'Connection': 'keep-alive',
    'Referer': 'http://www.iachina.cn/col/col41/?uid=1508&pageNum=1',
}

params = (
    ('startrecord', '1'),
    ('endrecord', '107'),
    ('perpage', '40'),
)

data = {
  'col': '1',
  'appid': '1',
  'webid': '1',
  'path': '/',
  'columnid': '41',
  'sourceContentType': '8',
  'unitid': '1508',
  'webname': '\u4E2D\u56FD\u4FDD\u9669\u884C\u4E1A\u534F\u4F1A',
  'permissiontype': '0'
}

response = requests.post('http://www.iachina.cn/module/web/jpage/dataproxy.jsp', headers=headers, params=params, cookies=cookies, data=data)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.post('http://www.iachina.cn/module/web/jpage/dataproxy.jsp?startrecord=1&endrecord=107&perpage=40', headers=headers, cookies=cookies, data=data)

#response = requests.post('http://www.iachina.cn/module/web/jpage/dataproxy.jsp', headers=headers, params=params, cookies=cookies, data=data, verify=False)
soup = BeautifulSoup(response.text, 'lxml')
chapter_list = soup.find_all(name='a')
all_chapter_url = [f"http://www.iachina.cn{i.get('href')}" for i in chapter_list]
all_chapter_text = [i.text for i in chapter_list]


In [3]:
pd.DataFrame({'url':all_chapter_url,
              'text':all_chapter_text})

,url,text
0,http://www.iachina.cn/art/2018/4/3/art_617_102...,2018年1-2月保险业经营情况表
1,http://www.iachina.cn/art/2018/3/9/art_617_102...,2018年1月保险业经营情况表
2,http://www.iachina.cn/art/2018/3/6/art_617_102...,2017年保险业经营情况表
3,http://www.iachina.cn/art/2018/1/10/art_617_10...,2017年1-11月保险业经营情况表
4,http://www.iachina.cn/art/2017/11/27/art_617_9...,2017年1-10月保险业经营情况表
...,...,...
102,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-5月保险业经营数据
103,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-4月保险业经营数据
104,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-3月保险业经营数据
105,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-2月保险业经营数据


In [ ]:
def clean_column(columns_list):
    """
    这个代码负责将表格的列名称清洗干净（就是将一些二级标题、三级标题都转换成完整的名称：{一级标题}-{二级标题}-{三级标题})
    如果标题是一级标题 -> 一级标题
    如果标题是二级标题 -> {一级标题}-{二级标题}
    如果标题是三级标题 -> {一级标题}-{二级标题}-{三级标题}

    这么一做，每一个表的列名称就不会混淆

    """

    column_data = pd.DataFrame({'column_name':[i.replace(" ", '') for i in columns_list]})
    # 做一级填充
    column_data['class_1'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='[0-9]+、', string=x)) !=1)#[len(re.findall(pattern='[0-9]+、', string=temp_column)) !=1 for temp_column in columns_list]
    column_data['class_1_name'] = column_data['column_name']
    column_data.loc[column_data['class_1']== False, 'class_1_name'] = np.nan
    column_data['class_1_name'] = column_data['class_1_name'].ffill()
    # column_data.loc[column_data['column_name'] != column_data['class_1_name'],'class_1_name'] = \
    #     column_data.loc[column_data['column_name'] != column_data['class_1_name'],:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    column_data.loc[column_data['class_1'] == False,'class_1_name'] = \
        column_data.loc[column_data['class_1'] == False,:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    # 做二级填充

    column_data['class_2'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='（[0-9]+）', string=x))!=1)

    column_data['class_2_name'] = column_data['class_1_name']
    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = np.nan
    column_data['class_2_name'] = column_data['class_2_name'].ffill()

    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = column_data.loc[column_data['class_2'] == False, :].apply(lambda x: f"{x['class_2_name']}__{x['column_name']}", axis=1)

    return column_data['class_2_name'].tolist()



def get_chapter_data(index):
    # time.sleep(random.randint(0, 1.5))

    temp_url = all_chapter_url[index]
    temp_info = all_chapter_text[index]
    data = pd.read_html(temp_url)[2].dropna()
    data = data.loc[data[0] != data[1], :]
    data = data.set_index(0).T.reset_index(drop=True)

    columns_list = data.columns.tolist()
    data.columns = clean_column(columns_list)

    # data.columns = [f"{index}_{columns_list[index]}" for index in range(len(columns_list))]


    data['from_url'] = temp_url
    data['info'] = temp_info
    return data

# test function
test_data = get_chapter_data(index=70)
test_data

In [ ]:
finaldata = pd.concat([get_chapter_data(index) for index in tqdm(range(len(all_chapter_url)))]).reset_index(drop=True)
finaldata.head(10)

In [17]:
finaldata.to_csv("保险业经营数据.csv", index=False)

In [ ]:
finaldata

## Jiangda Test